In [1]:
import psycopg2
import pandas as pd
import psycopg2.extras

In [38]:
item_dim = pd.read_excel (r'C:\Users\HP\dataset\e-commerece_data.xlsx', sheet_name='Item_dim')
item_dim.head(10)

,item_key,item_name,desc,unit_price,man_country,supplier,unit
0,I00001,A&W Root Beer - 12 oz cans,a. Beverage - Soda,11.50,Netherlands,Bolsius Boxmeer,cans
1,I00002,A&W Root Beer Diet - 12 oz cans,a. Beverage - Soda,6.75,poland,CHROMADURLIN S.A.S,cans
2,I00003,Barq's Root Beer - 12 oz cans,a. Beverage - Soda,6.75,Bangladesh,DENIMACH LTD,cans
3,I00004,Cherry Coke 12oz,a. Beverage - Soda,6.75,Netherlands,Bolsius Boxmeer,cans
4,I00005,Cherry Coke Zero 12 pack,a. Beverage - Soda,6.75,Finland,HARDFORD AB,cans
5,I00006,Coke Classic 12 oz cans,a. Beverage - Soda,16.25,Lithuania,BIGSO AB,cans
6,I00007,Coke Classic 12 oz cans,a. Beverage - Soda,6.75,India,Indo Count Industries Ltd,cans
7,I00008,Coke Zero Sugar 12 oz cans,a. Beverage - Soda,16.25,India,Indo Count Industries Ltd,cans
8,I00009,Diet Coke - 12 oz cans,a. Beverage - Soda,16.25,Netherlands,Bolsius Boxmeer,cans
9,I00010,Diet Coke - 12 oz cans,a. Beverage - Soda,6.75,Lithuania,BIGSO AB,cans


In [39]:
# Fill missing values with the most frequent value
item_dim = item_dim.fillna(item_dim.mode().iloc[0])
item_dim.head(10)

,item_key,item_name,desc,unit_price,man_country,supplier,unit
0,I00001,A&W Root Beer - 12 oz cans,a. Beverage - Soda,11.50,Netherlands,Bolsius Boxmeer,cans
1,I00002,A&W Root Beer Diet - 12 oz cans,a. Beverage - Soda,6.75,poland,CHROMADURLIN S.A.S,cans
2,I00003,Barq's Root Beer - 12 oz cans,a. Beverage - Soda,6.75,Bangladesh,DENIMACH LTD,cans
3,I00004,Cherry Coke 12oz,a. Beverage - Soda,6.75,Netherlands,Bolsius Boxmeer,cans
4,I00005,Cherry Coke Zero 12 pack,a. Beverage - Soda,6.75,Finland,HARDFORD AB,cans
5,I00006,Coke Classic 12 oz cans,a. Beverage - Soda,16.25,Lithuania,BIGSO AB,cans
6,I00007,Coke Classic 12 oz cans,a. Beverage - Soda,6.75,India,Indo Count Industries Ltd,cans
7,I00008,Coke Zero Sugar 12 oz cans,a. Beverage - Soda,16.25,India,Indo Count Industries Ltd,cans
8,I00009,Diet Coke - 12 oz cans,a. Beverage - Soda,16.25,Netherlands,Bolsius Boxmeer,cans
9,I00010,Diet Coke - 12 oz cans,a. Beverage - Soda,6.75,Lithuania,BIGSO AB,cans


In [40]:
item_dim.isnull().sum()

item_key       0
item_name      0
desc           0
unit_price     0
man_country    0
supplier       0
unit           0
dtype: int64

In [41]:
item_dim.rename(columns = {'desc':'descript'}, inplace = True)
item_dim.head(10)

,item_key,item_name,descript,unit_price,man_country,supplier,unit
0,I00001,A&W Root Beer - 12 oz cans,a. Beverage - Soda,11.50,Netherlands,Bolsius Boxmeer,cans
1,I00002,A&W Root Beer Diet - 12 oz cans,a. Beverage - Soda,6.75,poland,CHROMADURLIN S.A.S,cans
2,I00003,Barq's Root Beer - 12 oz cans,a. Beverage - Soda,6.75,Bangladesh,DENIMACH LTD,cans
3,I00004,Cherry Coke 12oz,a. Beverage - Soda,6.75,Netherlands,Bolsius Boxmeer,cans
4,I00005,Cherry Coke Zero 12 pack,a. Beverage - Soda,6.75,Finland,HARDFORD AB,cans
5,I00006,Coke Classic 12 oz cans,a. Beverage - Soda,16.25,Lithuania,BIGSO AB,cans
6,I00007,Coke Classic 12 oz cans,a. Beverage - Soda,6.75,India,Indo Count Industries Ltd,cans
7,I00008,Coke Zero Sugar 12 oz cans,a. Beverage - Soda,16.25,India,Indo Count Industries Ltd,cans
8,I00009,Diet Coke - 12 oz cans,a. Beverage - Soda,16.25,Netherlands,Bolsius Boxmeer,cans
9,I00010,Diet Coke - 12 oz cans,a. Beverage - Soda,6.75,Lithuania,BIGSO AB,cans


In [42]:
item_dim['descript'].replace("a."," ")

0      a. Beverage - Soda
1      a. Beverage - Soda
2      a. Beverage - Soda
3      a. Beverage - Soda
4      a. Beverage - Soda
              ...        
259      Kitchen Supplies
260      Kitchen Supplies
261      Kitchen Supplies
262              Medicine
263              Medicine
Name: descript, Length: 264, dtype: object

In [43]:
item_dim['descript'].str.split(".",expand=True)

,0,1
0,a,Beverage - Soda
1,a,Beverage - Soda
2,a,Beverage - Soda
3,a,Beverage - Soda
4,a,Beverage - Soda
...,...,...
259,Kitchen Supplies,None
260,Kitchen Supplies,None
261,Kitchen Supplies,None
262,Medicine,None


In [44]:
def extract_desc(input_text,level=0):
    output_text=input_text.split('.')
    output_text.fillna("a","")
    print(output_text) 


In [45]:
item_dim['descript']=item_dim['descript'].apply(extract_desc,level=0)

AttributeError: 'list' object has no attribute 'replace'

In [13]:
item_dim['descript']

0                     a
1                     a
2                     a
3                     a
4                     a
             ...       
259    Kitchen Supplies
260    Kitchen Supplies
261    Kitchen Supplies
262            Medicine
263            Medicine
Name: descript, Length: 264, dtype: object

In [15]:
item_dim.head(20)

,item_key,item_name,descript,unit_price,man_country,supplier,unit
0,I00001,A&W Root Beer - 12 oz cans,a,11.50,Netherlands,Bolsius Boxmeer,cans
1,I00002,A&W Root Beer Diet - 12 oz cans,a,6.75,poland,CHROMADURLIN S.A.S,cans
2,I00003,Barq's Root Beer - 12 oz cans,a,6.75,Bangladesh,DENIMACH LTD,cans
3,I00004,Cherry Coke 12oz,a,6.75,Netherlands,Bolsius Boxmeer,cans
4,I00005,Cherry Coke Zero 12 pack,a,6.75,Finland,HARDFORD AB,cans
5,I00006,Coke Classic 12 oz cans,a,16.25,Lithuania,BIGSO AB,cans
6,I00007,Coke Classic 12 oz cans,a,6.75,India,Indo Count Industries Ltd,cans
7,I00008,Coke Zero Sugar 12 oz cans,a,16.25,India,Indo Count Industries Ltd,cans
8,I00009,Diet Coke - 12 oz cans,a,16.25,Netherlands,Bolsius Boxmeer,cans
9,I00010,Diet Coke - 12 oz cans,a,6.75,Lithuania,BIGSO AB,cans
